<h3>torch.autograd</h3> is PyTorch’s automatic differentiation engine that powers neural network training. In this section, you will get a conceptual understanding of how autograd helps a neural network train.

*<h3>Background</h3>*<br>
Neural networks (NNs) are a collection of nested functions that are executed on some input data. These functions are defined by parameters (consisting of weights and biases), which in PyTorch are stored in tensors.

Training a NN happens in two steps:

Forward Propagation: In forward prop, the NN makes its best guess about the correct output. It runs the input data through each of its functions to make this guess.

Backward Propagation: In backprop, the NN adjusts its parameters proportionate to the error in its guess. It does this by traversing backwards from the output, collecting the derivatives of the error with respect to the parameters of the functions (gradients), and optimizing the parameters using gradient descent. For a more detailed walkthrough of backprop.

<h1>Usage in PyTorch</h1> <br>
Let’s take a look at a single training step. For this example, we load a pretrained resnet18 model from torchvision. We create a random data tensor to represent a single image with 3 channels, and height & width of 64, and its corresponding label initialized to some random values.

In [10]:
import torch, torchvision
model = torchvision.models.resnext50_32x4d(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Next, we run the input data through the model through each of its layers to make a prediction. This is the forward pass.

In [11]:
prediction = model(data) #forward pass

We use the model’s prediction and the corresponding label to calculate the error (loss). The next step is to backpropagate this error through the network. Backward propagation is kicked off when we call .backward() on the error tensor. Autograd then calculates and stores the gradients for each model parameter in the parameter’s .grad attribute.

In [12]:
loss = (prediction - labels).sum()
loss.backward()  #backward pass

In [13]:
optim = torch.optim.SGD(model.parameters(),lr=1e-2,momentum=0.9)

Finally, we call .step() to initiate gradient descent. The optimizer adjusts each parameter by its gradient stored in .grad.

In [14]:
optim.step() #gradient Descent

At this point, you have everything you need to train your neural network. The below sections detail the workings of autograd

<h2>Differentiation in Autograd</h2><br>
Let’s take a look at how autograd collects gradients. We create two tensors a and b with requires_grad=True. This signals to autograd that every operation on them should be tracked.

In [15]:
a=torch.tensor(2.0,requires_grad=True)
b= torch.tensor(6.0,requires_grad=True)

In [16]:
a+b,a-b,a*b,b/a


(tensor(8., grad_fn=<AddBackward0>),
 tensor(-4., grad_fn=<SubBackward0>),
 tensor(12., grad_fn=<MulBackward0>),
 tensor(3., grad_fn=<DivBackward0>))

We create another tensor Q from a and b.

               Q=3a3−b2

In [17]:
Q = 3*a**3 - b**2

Let’s assume a and b to be parameters of an NN, and Q to be the error. In NN training, we want gradients of the error w.r.t. parameters, i.e.

∂Q/∂a=9a2<br>
∂Q./b=−2b<br>
When we call .backward() on Q, autograd calculates these gradients and stores them in the respective tensors’ .grad attribute.

We need to explicitly pass a gradient argument in Q.backward() because it is a vector. gradient is a tensor of the same shape as Q, and it represents the gradient of Q w.r.t. itself, i.e.

dQ/dQ1<br>
Equivalently, we can also aggregate Q into a scalar and call backward implicitly, like Q.sum().backward().



In [18]:
Q.backward()

In [19]:
a.grad.data

tensor(36.)

In [20]:
b.grad.data

tensor(-12.)

In [21]:
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor(True)
tensor(True)
